In [1]:
import os
# Move working directory 1 level up
os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))
print("New working directory:", os.getcwd())

!pip install -r requirements.txt --quiet

New working directory: g:\Other computers\Flip15\UWA\3rd Semester\Data Warehousing\PROJECT WH\DataWH_project


#### 1. EXTRACT

In [2]:
os.path.abspath(os.getcwd())

'g:\\Other computers\\Flip15\\UWA\\3rd Semester\\Data Warehousing\\PROJECT WH\\DataWH_project'

In [3]:
from pathlib import Path
import pandas as pd

def extract_excel(file_name , sheet_name):
    # Extracting path of the repository
    BASE_DIR = Path.cwd().resolve()
    # Handle the extraction of excel data
    PATH = BASE_DIR / "raw_data" / file_name
    print(PATH)
    return pd.read_excel(PATH, sheet_name= sheet_name)

# Extracting data from the excel files, we will ignore all files that not been choisen to be used in the data warehouse
Fatal_Crash = extract_excel("bitre_fatal_crashes_dec2024.xlsx", "BITRE_Fatal_Crash")
Fatality = extract_excel("bitre_fatalities_dec2024.xlsx", "BITRE_Fatality")
Population = extract_excel("Population estimates by LGA, Significant Urban Area, Remoteness Area, Commonwealth Electoral Division and State Electoral Division, 2001 to 2023.xlsx", "Table 3")

G:\Other computers\Flip15\UWA\3rd Semester\Data Warehousing\PROJECT WH\DataWH_project\raw_data\bitre_fatal_crashes_dec2024.xlsx
G:\Other computers\Flip15\UWA\3rd Semester\Data Warehousing\PROJECT WH\DataWH_project\raw_data\bitre_fatalities_dec2024.xlsx
G:\Other computers\Flip15\UWA\3rd Semester\Data Warehousing\PROJECT WH\DataWH_project\raw_data\Population estimates by LGA, Significant Urban Area, Remoteness Area, Commonwealth Electoral Division and State Electoral Division, 2001 to 2023.xlsx


#### 2. TRANSFORMATION

In [4]:
def missing_treatment(data):
    # Handle the missing data
    data.replace("-9", "Unknown", inplace=True)
    data.replace(-9, "Unknown", inplace=True)
    #data.replace("Unknown", pd.NA, inplace=True)
    data.replace("-9/Unknown", "Unknown", inplace=True)
    return data

def clear_not_data_rows(data, n_rows):
    # Clearning the unwanted rows
    data.columns = data.iloc[n_rows]
    data.drop(index=range(0,n_rows+1), inplace=True) 
    # Reset the index after dropping rows
    data.reset_index(drop=True, inplace=True)
    return data

def unpivot_unmerge_cell(data):
    data["State/Territory"].fillna(method='ffill', inplace=True)
    # Unpivot the population data to long format
    data = data.melt(
        id_vars=[data.columns[0], data.columns[1]],  
        var_name="Year",      # col names to value col name
        value_name="Population"  # value contain col
    )
    return data

def extract_hour(data):
    # Convert the 'Time' column to datetime format and extract the hour as HH
    data['Time'] = pd.to_datetime(data['Time'], format='%H:%M:%S', errors='coerce') \
                    .dt.strftime('%H')
    return data

def create_sk(data, name_sk):
    # Create a surrogate key (sk) for the data
    data = data.reset_index(drop=True).reset_index(drop=False) # create sk
    data = data.rename(columns={'index': name_sk}) # change to sk
    data.iloc[:,0] = data.iloc[:,0]+1 # +1 for sk
    return data

def extract_drop_dup(data, list_col):
    # Extract the data and drop the duplicates
    data = data[list_col]
    data = data.drop_duplicates()
    return data

def create_dim_table(data, list_col, name_sk):
    # Pre-loading the data
    data = extract_drop_dup(data, list_col)
    data = create_sk(data, name_sk)
    return data

In [5]:
# Exploring data set Fatal_Crash
Fatal_Crash

,Fatal Crash Data,Unnamed: 1,(Data current to December 2024),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Note: A value of '-9' is used for a missing/un...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,National Remoteness Areas,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period,Day of week,Time of Day
4,20241115,NSW,12,2024,Friday,04:00:00,Single,1,No,No,No,100,Inner Regional Australia,Riverina,Wagga Wagga,Arterial Road,Yes,No,Weekday,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51283,19891246,NSW,1,1989,Wednesday,17:05:00,Single,1,Yes,-9,No,60,Unknown,NaN,NaN,Undetermined,No,No,Weekday,Day
51284,19892038,Vic,1,1989,Wednesday,18:50:00,Single,1,Yes,No,No,60,Unknown,NaN,NaN,Undetermined,No,No,Weekday,Night
51285,19894064,SA,1,1989,Wednesday,19:00:00,Multiple,1,No,-9,No,100,Unknown,NaN,NaN,Undetermined,No,No,Weekday,Night
51286,19896006,Tas,1,1989,Wednesday,20:20:00,Multiple,6,No,-9,Yes,100,Unknown,NaN,NaN,Undetermined,No,No,Weekday,Night


In [6]:
# Clearing the data from the Fatal_Crash
Fatal_Crash = clear_not_data_rows(Fatal_Crash, 3)
Fatal_Crash = missing_treatment(Fatal_Crash)
Fatal_Crash = extract_hour(Fatal_Crash)
# Rename the column that have \n in its name
Fatal_Crash = Fatal_Crash.rename(columns={'Bus \nInvolvement': 'Bus Involvement'})
Fatal_Crash

C:\Users\dtt16\AppData\Local\Temp\ipykernel_32204\3785816588.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace(-9, "Unknown", inplace=True)


3,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,National Remoteness Areas,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period,Day of week,Time of Day
0,20241115,NSW,12,2024,Friday,04,Single,1,No,No,No,100,Inner Regional Australia,Riverina,Wagga Wagga,Arterial Road,Yes,No,Weekday,Night
1,20241125,NSW,12,2024,Friday,06,Single,1,No,No,No,80,Inner Regional Australia,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Local Road,No,No,Weekday,Day
2,20246013,Tas,12,2024,Friday,09,Multiple,1,No,No,No,50,Inner Regional Australia,Launceston and North East,Northern Midlands,Local Road,Yes,No,Weekday,Day
3,20241002,NSW,12,2024,Friday,10,Multiple,1,No,No,No,100,Outer Regional Australia,New England and North West,Armidale Regional,National or State Highway,No,No,Weekday,Day
4,20242261,Vic,12,2024,Friday,11,Multiple,1,Unknown,Unknown,Unknown,Unknown,Unknown,NaN,NaN,Undetermined,No,No,Weekday,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51279,19891246,NSW,1,1989,Wednesday,17,Single,1,Yes,Unknown,No,60,Unknown,NaN,NaN,Undetermined,No,No,Weekday,Day
51280,19892038,Vic,1,1989,Wednesday,18,Single,1,Yes,No,No,60,Unknown,NaN,NaN,Undetermined,No,No,Weekday,Night
51281,19894064,SA,1,1989,Wednesday,19,Multiple,1,No,Unknown,No,100,Unknown,NaN,NaN,Undetermined,No,No,Weekday,Night
51282,19896006,Tas,1,1989,Wednesday,20,Multiple,6,No,Unknown,Yes,100,Unknown,NaN,NaN,Undetermined,No,No,Weekday,Night


In [7]:
# Exploring data set Fatality
Fatality

,Fatality Data,Unnamed: 1,(Data current to December 2024),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Note: A value of '-9' is used for a missing/un...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,...,Age,National Remoteness Areas,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period,Age Group,Day of week,Time of day
4,20241115,NSW,12,2024,Friday,04:00:00,Single,No,No,No,...,74,Inner Regional Australia,Riverina,Wagga Wagga,Arterial Road,Yes,No,65_to_74,Weekday,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56873,19896006,Tas,1,1989,Wednesday,20:20:00,Multiple,No,-9,Yes,...,13,Unknown,NaN,NaN,Undetermined,No,No,0_to_16,Weekday,Night
56874,19896006,Tas,1,1989,Wednesday,20:20:00,Multiple,No,-9,Yes,...,13,Unknown,NaN,NaN,Undetermined,No,No,0_to_16,Weekday,Night
56875,19896006,Tas,1,1989,Wednesday,20:20:00,Multiple,No,-9,Yes,...,18,Unknown,NaN,NaN,Undetermined,No,No,17_to_25,Weekday,Night
56876,19896006,Tas,1,1989,Wednesday,20:20:00,Multiple,No,-9,Yes,...,14,Unknown,NaN,NaN,Undetermined,No,No,0_to_16,Weekday,Night


In [8]:
# Clearing the data from the Fatal_Crash
Fatality = clear_not_data_rows(Fatality, 3)
Fatality = missing_treatment(Fatality)
Fatality = extract_hour(Fatality)
Fatality

C:\Users\dtt16\AppData\Local\Temp\ipykernel_32204\3785816588.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace(-9, "Unknown", inplace=True)


3,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,...,Age,National Remoteness Areas,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period,Age Group,Day of week,Time of day
0,20241115,NSW,12,2024,Friday,04,Single,No,No,No,...,74,Inner Regional Australia,Riverina,Wagga Wagga,Arterial Road,Yes,No,65_to_74,Weekday,Night
1,20241125,NSW,12,2024,Friday,06,Single,No,No,No,...,19,Inner Regional Australia,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Local Road,No,No,17_to_25,Weekday,Day
2,20246013,Tas,12,2024,Friday,09,Multiple,No,No,No,...,33,Inner Regional Australia,Launceston and North East,Northern Midlands,Local Road,Yes,No,26_to_39,Weekday,Day
3,20241002,NSW,12,2024,Friday,10,Multiple,No,No,No,...,32,Outer Regional Australia,New England and North West,Armidale Regional,National or State Highway,No,No,26_to_39,Weekday,Day
4,20242261,Vic,12,2024,Friday,11,Multiple,Unknown,Unknown,Unknown,...,62,Unknown,NaN,NaN,Undetermined,No,No,40_to_64,Weekday,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56869,19896006,Tas,1,1989,Wednesday,20,Multiple,No,Unknown,Yes,...,13,Unknown,NaN,NaN,Undetermined,No,No,0_to_16,Weekday,Night
56870,19896006,Tas,1,1989,Wednesday,20,Multiple,No,Unknown,Yes,...,13,Unknown,NaN,NaN,Undetermined,No,No,0_to_16,Weekday,Night
56871,19896006,Tas,1,1989,Wednesday,20,Multiple,No,Unknown,Yes,...,18,Unknown,NaN,NaN,Undetermined,No,No,17_to_25,Weekday,Night
56872,19896006,Tas,1,1989,Wednesday,20,Multiple,No,Unknown,Yes,...,14,Unknown,NaN,NaN,Undetermined,No,No,0_to_16,Weekday,Night


In [9]:
# Exploring data set Population
Population

,This tab has one table with the estimated resident population for Remoteness Areas in Australia as at 30 June 2023. It ranges from cell A1 to Y68.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,Australian Bureau of Statistics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Table 3. Estimated resident population, Remote...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Regional population, 2022-23",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,ERP at 30 June,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
5,State/Territory,Remoteness Area,no.,no.,no.,no.,no.,no.,no.,no.,...,no.,no.,no.,no.,no.,no.,no.,no.,no.,no.
6,New South Wales,Major Cities of Australia (NSW),4791646,4832878,4867228,4894766,4931770,4974013,5052320,5146661,...,5627414,5723253,5825821,5930874,6011991,6088337,6137535,6112572,6166301,6324725
7,NaN,Inner Regional Australia (NSW),1309389,1321149,1330957,1337690,1346082,1354983,1369615,1384368,...,1465979,1478446,1492516,1509682,1527522,1543589,1559150,1571760,1586980,1602106
8,NaN,Outer Regional Australia (NSW),384554,382991,379956,376856,374896,374139,373071,373575,...,377413,377496,378057,378749,379438,379860,379593,378970,380023,382007
9,NaN,Remote Australia (NSW),34756,34075,33193,32316,31589,30974,30675,30413,...,29496,29038,28670,28378,28076,27657,27202,26778,26549,26581


In [10]:
# Transform the Population data
column_names = Population.iloc[5]
# Handle the missing data
Population = clear_not_data_rows(Population, 5)
# Change the column names with years
Population.columns = list(column_names[:2]) + [f"{year}" for year in range(2001, 2024)]
# Drop the last row that is row of notes
Population.drop(index=Population.index[-1], inplace=True)
Population = unpivot_unmerge_cell(Population)

# Taking only rows with "Total" in Remoteness Area for the state population and with Year = 2023
Population = Population[
    Population["Remoteness Area"].str.contains("Total", case=False, na=False) &
    (Population["Year"] == "2023")
].copy() 

# Drop the 'Remoteness Area' and 'Year' column as it is no longer needed
Population = Population.drop(columns=["Remoteness Area", "Year"])
# Rename column "State/Territory" to "State"
Population = Population.rename(columns={"State/Territory": "State"})

# Replace full state names with abbreviations
Population["state_name"] = Population["State"]
state_map = {
    "New South Wales": "NSW",
    "Victoria": "VIC",
    "Queensland": "QLD",
    "South Australia": "SA",
    "Western Australia": "WA",
    "Tasmania": "TAS",
    "Northern Territory": "NT",
    "Australian Capital Territory": "ACT",
    "Other Territories": "Other",
    "Australia" : "AUS"
}
Population["State"] = Population["State"].replace(state_map).copy()

Population

C:\Users\dtt16\AppData\Local\Temp\ipykernel_32204\3785816588.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["State/Territory"].fillna(method='ffill', inplace=True)
C:\Users\dtt16\AppData\Local\Temp\ipykernel_32204\3785816588.py:18: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["State/Territory"].fillna(method='ffill', inplace=True)


,State,Population,state_name
1105,NSW,8342285,New South Wales
1110,VIC,6815441,Victoria
1116,QLD,5460420,Queensland
1122,SA,1852284,South Australia
1128,WA,2881227,Western Australia
1133,TAS,573156,Tasmania
1137,NT,252529,Northern Territory
1140,ACT,466566,Australian Capital Territory
1143,Other,4970,Other Territories
1149,AUS,26648878,Australia


In [11]:
# Creating the dim_crash_group table
list_col = ['Crash Type', 'Speed Limit', 'National Road Type', 'Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement']
dim_crash_group = create_dim_table(Fatal_Crash, list_col, "crashgroup_sk")
dim_crash_group

3,crashgroup_sk,Crash Type,Speed Limit,National Road Type,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement
0,1,Single,100,Arterial Road,No,No,No
1,2,Single,80,Local Road,No,No,No
2,3,Multiple,50,Local Road,No,No,No
3,4,Multiple,100,National or State Highway,No,No,No
4,5,Multiple,Unknown,Undetermined,Unknown,Unknown,Unknown
...,...,...,...,...,...,...,...
577,578,Multiple,20,Undetermined,Yes,Unknown,No
578,579,Single,75,Undetermined,No,No,Yes
579,580,Single,30,Undetermined,Yes,Unknown,No
580,581,Multiple,110,Undetermined,Yes,Unknown,Yes


In [12]:
# Creating the dim_fatality_demographic table
list_col = ['Road User', 'Gender', 'Age Group']
dim_fatality_demographic = create_dim_table(Fatality, list_col, "fatality_demographics_sk")
dim_fatality_demographic

3,fatality_demographics_sk,Road User,Gender,Age Group
0,1,Driver,Male,65_to_74
1,2,Driver,Female,17_to_25
2,3,Driver,Female,26_to_39
3,4,Passenger,Male,40_to_64
4,5,Passenger,Female,40_to_64
...,...,...,...,...
102,103,Motorcycle rider,Female,0_to_16
103,104,Motorcycle pillion passenger,Female,65_to_74
104,105,Driver,Unknown,0_to_16
105,106,Motorcycle pillion passenger,Female,Unknown


In [13]:
# Creating the dim_fatality_demographic table
list_col = ['Time','Dayweek','Month','Year']
dim_time = create_dim_table(Fatal_Crash, list_col, "time_sk")
dim_time

3,time_sk,Time,Dayweek,Month,Year
0,1,04,Friday,12,2024
1,2,06,Friday,12,2024
2,3,09,Friday,12,2024
3,4,10,Friday,12,2024
4,5,11,Friday,12,2024
...,...,...,...,...,...
34752,34753,17,Wednesday,1,1989
34753,34754,18,Wednesday,1,1989
34754,34755,19,Wednesday,1,1989
34755,34756,20,Wednesday,1,1989


In [14]:
# Dimension 'dim_location'
dim_location = Fatal_Crash[['State']].copy()
# Remove duplicate rows
dim_location = dim_location.drop_duplicates().reset_index(drop=True)
# Drop all rows that dont have LGA name, which is the last row of the table
dim_location.drop(index=dim_location[dim_location['State'].isna()].index, inplace=True)
# Uppercase the state names
dim_location['State'] = dim_location['State'].str.upper()
# Join the population data with the location data
dim_location = dim_location.merge(Population, how='left', left_on='State', right_on='State')
dim_location

,State,Population,state_name
0,NSW,8342285,New South Wales
1,TAS,573156,Tasmania
2,VIC,6815441,Victoria
3,QLD,5460420,Queensland
4,SA,1852284,South Australia
5,WA,2881227,Western Australia
6,ACT,466566,Australian Capital Territory
7,NT,252529,Northern Territory


In [15]:
# Create the fact table 'fact_crash'
# Left outer join all dims into the main data 'Fatality' - Data of each fatality
joinOn = list(dim_crash_group.columns[1:])
Fatality = Fatality.merge(dim_crash_group, on=joinOn, how="left")

joinOn = list(dim_fatality_demographic.columns[1:])
Fatality = Fatality.merge(dim_fatality_demographic, on=joinOn, how="left")

joinOn = list(dim_time.columns[1:])
Fatality = Fatality.merge(dim_time, on=joinOn, how="left")

Fatality["State"] = Fatality["State"].str.upper()
Fatality = Fatality.merge(dim_location, on="State", how="left")

# Extract attributes for fact table
fact_crash = Fatality[['Crash ID','crashgroup_sk','fatality_demographics_sk', 'time_sk', 'State']].copy()
fact_crash.columns = ['crash_id', 'crashgroup_sk', 'fatality_demographics_sk', 'time_sk', 'state_id']
# Surrogate key creating by index
fact_crash = fact_crash.reset_index(drop=True) # Reset for correct indexing
fact_crash = fact_crash.reset_index(drop=False)
fact_crash = fact_crash.rename(columns={'index': 'crash_sk'})
fact_crash.iloc[:,0] = fact_crash.iloc[:,0]+1 
fact_crash

,crash_sk,crash_id,crashgroup_sk,fatality_demographics_sk,time_sk,state_id
0,1,20241115,1,1,1,NSW
1,2,20241125,2,2,2,NSW
2,3,20246013,3,3,3,TAS
3,4,20241002,4,3,4,NSW
4,5,20242261,5,4,5,VIC
...,...,...,...,...,...,...
56869,56870,19896006,529,48,34756,TAS
56870,56871,19896006,529,43,34756,TAS
56871,56872,19896006,529,15,34756,TAS
56872,56873,19896006,529,48,34756,TAS


In [16]:
# Rename the columns of the dimension tables
dim_crash_group.columns = ['crashgroup_sk','crash_type', 'speed_limit', 'road_type', 'bus_involvement', 'heavy_rigid_truck_involvement', 'articulated_truck_involvement']
dim_fatality_demographic.columns = ['fatality_demographics_sk','road_user', 'gender', 'age_group']
dim_time.columns = ['time_sk', 'time', 'dayweek', 'month', 'year']
dim_location.columns = ['state_id','population','state_name']

## 3. Load data to data warehouse

In [17]:
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# Load environment variables from .env file, remember to create this file with your credentials
# Read readme file for more information ! 
load_dotenv("postgressql_secretkeys.env")  
# Extract variables
username = os.getenv("PG_USER")
password = os.getenv("PG_PASSWORD")
host     = os.getenv("PG_HOST")
port     = os.getenv("PG_PORT")
dbname   = os.getenv("PG_DB")

# Create the SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}")

# Test connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT version();"))
    print(result.fetchone())
    print("======================================= \n || -- Connected to PostgreSQL! -- || \n=======================================")


('PostgreSQL 17.4 on x86_64-windows, compiled by msvc-19.42.34436, 64-bit',)
 || -- Connected to PostgreSQL! -- || 


In [18]:
from sqlalchemy import create_engine, text

# Read SQL file
with open("ETL/create_schema.sql", "r", encoding="utf-8") as f:
    schema_sql = f.read()

# Execute SQL script
with engine.connect() as conn:
    conn.execute(text(schema_sql))
    conn.commit()
    print("======================================= \n || -- Schema Created! -- || \n=======================================")

 || -- Schema Created! -- || 


In [19]:
#Main: Load to Data Warehouse
dim_crash_group.to_sql("dim_crash_group", engine, if_exists="append", index=False)
dim_fatality_demographic.to_sql("dim_fatality_demographic", engine, if_exists="append", index=False)
dim_time.to_sql("dim_time", engine, if_exists="append", index=False)
dim_location.to_sql("dim_location", engine, if_exists="append", index=False)
fact_crash.to_sql("fact_crash", engine, if_exists="append", index=False)

print("======================================= \n || -- Data Loaded to PostgreSQL! -- || \n=======================================")

 || -- Data Loaded to PostgreSQL! -- || 


In [20]:
from pathlib import Path

# Optional: Export to csv file
create_local_csv = False  # FLAG!: Set to True if you want to export to csv files

if create_local_csv:
    output_dir = Path("raw_data")
    output_dir.mkdir(parents=True, exist_ok=True)  # Ensure the folder exists

    dim_crash_group.to_csv(output_dir / "dim_crash_group.csv", index=False)
    dim_fatality_demographic.to_csv(output_dir / "dim_fatality_demographic.csv", index=False)
    dim_time.to_csv(output_dir / "dim_time.csv", index=False)
    dim_location.to_csv(output_dir / "dim_location.csv", index=False)
    fact_crash.to_csv(output_dir / "fact_crash.csv", index=False)
    print("======================================= \n || -- CSV Files Created! -- || \n=======================================")

In [22]:
# This is just a funny melody to play when the script is finished
# This to let me know that the script is finished and I can do something else
import winsound
import time

# Short funny retro-style melody
notes = [
    (659, 150),  # E5
    (784, 150),  # G5
    (880, 150),  # A5
    (659, 300),  # E5
    (523, 300),  # C5
    (587, 300),  # D5
    (659, 600),  # E5 long
]
for freq, dur in notes:
    winsound.Beep(freq, dur)
    time.sleep(0.05)  
for freq, dur in notes:
    winsound.Beep(freq, dur)
    time.sleep(0.05)  
